# [OpenStreetMap](http://www.openstreetmap.org/) - A Data Cleaning Case Study

If you're wondering what OpenStreetMap is, think of it as Wikipedia for meta-data-filled maps of the world. From the [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page):

> "Welcome to OpenStreetMap, the project that creates and distributes free geographic data for the world. We started it because most maps you think of as free actually have legal or technical restrictions on their use, holding back people from using them in creative, productive, or unexpected ways."

This case study should give any reader a sufficient introduction to basic XML parsing, Python scripting of fixes and SQL database queries of OpenStreetMap data.

### Focus of the Case Study:
According to [The New York Times' 'What Could Disappear'](http://www.nytimes.com/interactive/2012/11/24/opinion/sunday/what-could-disappear.html?_r=2&) interactive article, New Orleans could be 88% below sea level in as little as 100 years. I'm personally interested to see what we're set to lose as a consequence of global warming so I'll focus on approximately 100x150 mile area [surrounding New Orleans](https://mapzen.com/data/metro-extracts/metro/new-orleans_louisiana/).

![map area](supporting_files/screenshots/mapzen_extract_map.png "map area")

### Getting to know the data:
OpenStreetMap data is available as XML bearing the file type '.osm'. Reading through the [documentation](https://wiki.openstreetmap.org/wiki/Main_Page), I find the gist of osm data to be:
* A **'node'** element essentially represents a latitude/longitude coordinate. It may have 'tag' child elements with other data points comprising things like an 'address' or features such as 'school'.
* A **'way'** element has nodes as children (with their element type shortened to 'nd'). It defines things like roads, buildings, natural areas, etc. It also may have 'tag' children for the same reasons as a node.
* A **'relation'** has nested 'member' elements which reference existing ways and nodes. It defines logical or geographic relationships between other elements such as an extended hiking trail made up by a number of way elements.

The osm file unzips to a whopping 1.28gb and crashes both Atom and SublimeText on my machine. [VIM](http://www.vim.org/), on the other hand, is a wonderfully simple text editor designed to be used in a bare-bones command line interface (like the Terminal app on a mac). It can easily handle the job. I used a few basic VIM commands to jump around and explore the osm file freely:
* Jump to bottom: `shift+g`
* Jump to top: `gg`
* page down: `Control+d`
* page up: `Control+u`
* Jump five million lines down: type `5000000` and then hit `j`
* Jump five million lines up: type `5000000` and then hit `k`
* Search the entire massive file for anything you want: type `/` followed by your search
* Go to the next instance of what you searched for: `n`
    
*Side note: check out [this little gem](https://vim-adventures.com/) if you are interested in getting started with VIM in a fun way.*

Here is what I learned from exploring the file:
* The file has the standard opening:  <?xml version='1.0' encoding='UTF-8'?>. I'll use this later when creating a creating a smaller file-size sample osm document later in this project.
* The XML root element is 'osm' and is parent to all other elements
* There are 16,082,009 lines in total - wow!
* an example of a node element with no tags:

![node](supporting_files/screenshots/node.png "node")

* a node element with tags:

![node with tags](supporting_files/screenshots/node_tags.png "node with tags")

* a way element with its nested nd (node) element references and tags:

![way](supporting_files/screenshots/way.png "way")

* a relation element with its nested member elements that reference nodes and ways. It also has tag elements:

![relation](supporting_files/screenshots/relation.png "relation")

### Reflecting upon the data - what should I clean?:
Thinking out loud (writing actually), I can see that tag elements seem to constitute the bulk of a way or relation element's meta-data. From what I understand about OpenStreetMap, this should also be where a lot of user-generated/added content is stored. I'll probably have to parse entire elements in order to create a well-structured database of the data, but the focus of my cleaning may be in these tags. 

One other thing catches my eye - the large number of tag elements who's 'k' attribute contains the acronym 'NHD'. What is that about? It turns out NHD stands for the US Geological Survey's ['National Hydrography Dataset'](https://nhd.usgs.gov/), which maps out and documents the nation's watershed boundaries and their features. New Orleans is surrounded by a massive amount of such natural features. These will be among the first to go as ocean levels rise, and in my opinion it's worth knowing about it. This is my focus - the New Orleans watershed:


![new orleans watershed](supporting_files/screenshots/new_orleans_watershet.png "new orleans watershed")


I would assume the data to be damn near perfect given it was most likely adopted directly from the NHD. The wonderful thing here is that, without much trouble, I can programmatically verify whether or not that is true despite the over 16 million lines of data to deal with.

Ok, close your other browser tabs, put on some Mozart - here we go!!!

### Auditing the data:
In the tag elements I mentioned, 'NHD' stands out as a prefix for the field names of the NHD data (ex. 'ComID', 'FCode' and 'FTYPE'). I need to rely on one of these to isolate the elements in the osm file that I'm interested in when investigating or parsing. I used the official [NHD data model](https://nhd.usgs.gov/NHDv2.2.1_poster_081216.pdf) and the [NHD data dictionary](https://nhd.usgs.gov/userGuide/Robohelpfiles/NHD_User_Guide/Feature_Catalog/Data_Dictionary/Data_Dictionary.htm) to find that ComID is a unique identifier for a natural feature. That will work. 

---
I've created a number of executable Python scripts in the `supporting_files/scripts` folder accompanying this report. Running **keys_list.py** will use ComID as a filter criteria to show a list of all unique tag element 'k' attributes (key values) from the elements I'm focusing on:

---

Looking over the list I already see some possible redundancies in the field naming convention for some of the NHD keys. Investigating the NHD keys and listing out the problematic or noteworthy:

##### NHD:ComID:
The 'Model Changes' section of the NHD data model notes, "ComID field deleted from all feature classes/tables". It turns out that it has been replaced by a different field called 'Permanent_Identifier' as its unique identifier. The NHD data dictionary clarifies - "features already assigned a ComID retain that value as the Permanent_ Identifier". While this is an important note for future data entry, it will not require cleaning here.

##### NHD:FTYPE and NHD:FType
These two keys appear to be referring to the same thing. To check, I manually searched through the osm file with VIM (ex. `/NHD:FTYPE`). Indeed, both 'FTYPE' and 'FType' refer to the same category of things - 'SwampMarsh', 'StreamRiver', etc. According to the NHD data model, 'FType' is correct.

To fix (clean) this and anything else I may come across, I created a non-executable python file called fix_dict.py (supporting_files/scripts) that will be envoked in a later script I'll use to parse the data. The basic algorithm is very simple:
1. input python dictionary object with data from a tag element
2. if key value is 'FTYPE', overwrite with 'FType'

##### NHD:GNIS_ID
'GNIS' stands for the '[Geographic Names Information System](https://nhd.usgs.gov/gnis.html)', which is another data set of natural feature identifiers from the U.S. Geological Survey:

>"The Geographic Names Information System (GNIS), developed by the U.S. Geological Survey in cooperation with the U.S. Board on Geographic Names, contains information about physical and cultural geographic features in the United States and associated areas..."

VIM searching (`/NHD:GNIS_ID`) for instances of these in the osm file I found that the seemingly invariable share the save value (the 'v' atribute of a tag element) as another tag element with the key of 'gnis:feature_id'. If indeed this is a redundancy, the only thing I can do is to flag/identify every instance for later deletion from the source OpenStreetMap data set. I'll do this towards the end of the case study using SQL queries, after setting up the database with my parsed data.

##### NHD:Permanent_
Again manually browsing the osm file I find that this field, which I'm assuming refers to the NHD data model field 'Permanent_Identifier', almost invariably shares the save value as 'NHD:ComID'. From what I learned about the ComID, however, it would make sense to use them here as they may have been assigned before the change to the 40-character Permanent_Identifier field. Still, I'll use SQL later to find every instance in case this could be helpful to the OpenStreetMap project.

##### NHD:RESOLUTION
This is something I can fix using my `fix_dict.py` file. According to the NHD data model it should be 'Resolution'. While this seems minor, it still fulfills my goal to make the data more accurate and consistent.

#### NHD:way_id
This confused me at first as 'way_id' is not an official field in the NHD data model. I soon noticed that tags with this key have the same value as 'NHD:ComID'. This one seems not only redundant but misleading because its value has nothing to do with the id attribute for the parent way tag. As with some of the others, I'll use an SQL query to flag each of these instances.

I'm focusing on NHD data but I don't want to omit the other tag elements while I have the opportunity to easily examine them. Yet, there isn't an official source I can use to verify they are valid, accurate or complete. My best bet is to manually review them and use my best judgement. This is also an opportunity to review the values of the NHD data. 

---
Running **audit_keys.py** in the scripts folder will show each key and a list of all values contained within the scope of my data set. Starting from the top of the list, the standouts are:

---

##### NHD:ComID
Some 
It has more than one entry in some of them. I track down way 43261226, but nothing I can do here reallly.

##### NHD:FCode
Again, more than one entry, but this seems ok to have more than one. 


##### NHD:FDate
Dupes, nothing can do

##### NHD:FTYPE and FHD:FType
Will be merged 

##### NHD:ReachCode
* "Unique identifier composed of two parts, first eight digits = subbasin code as defined by FIPS 103, and next six digits = random-assigned sequential number unique within a Cataloguing Unit."
    * issues that I see but can't fix:
           * Is supposed to be a unique identifier but some have more than one. Manually inspecting one such element doesn't reveal why, but noticed also has two NHD:ComID values. (WayID: 43393226)
           
##### NHD:way_id 
not official, dupe of other data

##### name
Scanning through 'name' I can see there is just an inconsistency in the naming, where some have the first letter of each capitalized and others don't. I'll fix that.

##### natural
There appears to be a redundant value between '_coastline' and 'coastline'. A quick VIM search in the osm document for '_coastline' reveals a 'note' key in a 'tag' element reading "I have altered the natural:coatline tag as this way duplicates existing coastline ways". Not touching this one.

##### note
I'm not really sure what this is about so I'll write a little script to pull out all elements bearing a 'note' tag element to inspect:

RUN audit_notes.py

Not going to do anything about the parent or 'tag' child elements with 'note' as their key.

##### source
Fix, NHD & bing to NHD & Bing

##### water
Minor, but Bayou should be bayou.

### Parse, clean and compile database:

run data.py

So now that I've noted above how to fix each thing, here is how I'm going to do it (algos)

Next, here is how the data will be parsed, cleaning algos inserted, and database compiled (algo)

### Examine the data efficiently with database queries

First I build the database:

In [4]:
import sqlite3
import csv
from pprint import pprint
from supporting_files import database_schema as ds
import os

sqlite_file = 'supporting_files/exports_databases/osm_db.db'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [5]:
sql_schema = ds.sql_schema

In [6]:
# Create the table, specifying the schema
c.executescript(sql_schema)
# commit the changes
conn.commit()

In [7]:
# first row of CSVs MUST be fieldnames
import os
from csv import DictReader

def add_table_from_csv(filename):
    if not filename.endswith('.csv'):
        return
    
    tablename = filename.split(".")[0].split('/')[-1]
    with open(filename, 'r') as fin:
        dr = DictReader(fin)
        fieldnames = dr.fieldnames
        to_db = [tuple( [i[k] for k in dr.fieldnames] ) for i in dr]
    c.executemany("INSERT INTO {}({}) VALUES({}?);".format(
            tablename, ",".join(dr.fieldnames), "?," * (len(dr.fieldnames)-1)), to_db)
    conn.commit()
    
def add_to_db(directory):
    for f_name in os.listdir(directory):
        full_path = os.path.join(directory, f_name)
        add_table_from_csv(full_path)

add_to_db('supporting_files/exports_csv')

##### Investigate any other questions I may have

Database file size:

In [8]:
database_size = os.path.getsize(sqlite_file) / 1000000
print("The size of the database is: %s mb" %round(database_size, 2))

The size of the database is: 77.13 mb


Create my own function for rendering a database query into a pandas dataframe:

In [9]:
import pandas as pd
pd.set_option('display.max_rows', 6)
def df_query(query):
    df = pd.read_sql(query, conn)
    return df

Top 10 contributors:

In [10]:
query = """
SELECT
    all_tables.user,
    SUM(Total) Total
FROM
    (
    SELECT
        nodes.user,
        COUNT(*) AS Total
    FROM nodes
    GROUP BY nodes.user

    UNION ALL

    SELECT
        relations.user,
        COUNT(*) AS Total
    FROM relations
    GROUP BY relations.user
    
    UNION ALL
    
    SELECT
        ways.user,
        COUNT(*) AS Total
    FROM ways
    GROUP BY ways.user
    ORDER BY Total DESC   
    
    ) AS all_tables

GROUP BY user
ORDER BY Total DESC
LIMIT 10
"""

df_query(query)

,user,Total
0,Matt Toups,25243
1,Maarten Deen,17221
2,Andre68,4370
...,...,...
7,wvdp,455
8,eric22,367
9,DKNOTT,201


Total number of nodes, ways, relations:

In [11]:
# nodes
query = """
SELECT COUNT(*) AS Total FROM nodes
"""

df_query(query)

,Total
0,2


In [12]:
# ways
query = """
SELECT COUNT(*) AS Total FROM ways
"""

df_query(query)

,Total
0,53562


In [13]:
# relations
query = """
SELECT COUNT(*) AS Total FROM relations
"""

df_query(query)

,Total
0,406


In [14]:
# nodes, ways and relations
query = """
SELECT
    SUM(all_tables.Total) AS Total_All
FROM 
    (
        SELECT COUNT(*) AS Total FROM nodes

        UNION ALL

        SELECT COUNT(*) AS Total FROM ways

        UNION ALL

        SELECT COUNT(*) AS Total FROM relations
    ) AS all_tables
"""

df_query(query)

,Total_All
0,53970


Unnamed features:
* Way ID: 43829974 connects to one of its nodes: 555906935 which has lat/lon coordinates lat="29.2658771" lon="-89.4146036". OpenStreetMaps shows those coordinates indeeed to be just some random natural feature [hEre](http://www.openstreetmap.org/search?query=29.2658771%2C%20-89.4146036#map=18/29.26588/-89.41460).

What about all of the unnamed features?
First, to review the named ones:

In [15]:
# List of named features
query = """

SELECT
    all_tables.value
FROM
    (
    SELECT
        nodes_tags.value
    FROM nodes_tags
    WHERE nodes_tags.key="name"
    GROUP BY nodes_tags.value
    
    UNION ALL
    
    SELECT
        ways_tags.value
    FROM ways_tags
    WHERE ways_tags.key="name"
    GROUP BY ways_tags.value
    
    UNION ALL
    
    SELECT
        relations_tags.value
    FROM relations_tags
    WHERE relations_tags.key="name"
    GROUP BY relations_tags.value
    ORDER BY relations_tags.value
    
    ) AS all_tables

GROUP BY all_tables.value
ORDER BY all_tables.value
"""

df_query(query)

,value
0,Abita Creek
1,Abita River
2,Adema Pond
...,...
265,Yankee Pond
266,Yellow Bayou
267,Yellow Lake Bayou


Number of unnamed natural features:

In [16]:
# All node/way/relation ids with 'FType' as their key
query = """
SELECT
    all_tables.value,
    COUNT(*) AS Total
FROM
    (
    SELECT
        nodes_tags.value
    FROM nodes_tags
    WHERE nodes_tags.key="FType"
    
    UNION ALL
    
    SELECT
        ways_tags.value
    FROM ways_tags
    WHERE ways_tags.key="FType"
    
    UNION ALL
    
    SELECT
        relations_tags.value
    FROM relations_tags
    WHERE relations_tags.key="FType"
    
    ) AS all_tables
GROUP BY all_tables.value
ORDER BY Total DESC
"""

df_query(query)

,value,Total
0,LakePond,22278
1,SwampMarsh,16200
2,StreamRiver,12371
...,...,...
10,Gate,6
11,Pipeline,6
12,Wall,6


Note cleaning that needs to be done here - circle back to data.py algo and adjust where necessary... or,this is not cleaning - just the best way to describe the data...

All features that are not already named

In [17]:
# list of FTYPEs that are not in the list of named features
query = """
SELECT *, COUNT(*) as Total
    FROM (
    SELECT
        all_tables.id,
        all_tables.value
    FROM
        (
        SELECT
            *
        FROM nodes_tags
        WHERE nodes_tags.key="FType"
        
        UNION ALL
        
        SELECT
            *
        FROM ways_tags
        WHERE ways_tags.key="FType"

        UNION ALL

        SELECT
            *
        FROM relations_tags
        WHERE relations_tags.key="FType"

        ) AS all_tables
    ) AS ftype_table
WHERE ftype_table.id NOT IN (
    SELECT
        all_tables.id
    FROM
        (
        SELECT
            *
        FROM nodes_tags
        WHERE nodes_tags.key="name"
        
        UNION ALL
        
        SELECT
            *
        FROM ways_tags
        WHERE ways_tags.key="name"

        UNION ALL

        SELECT
            *
        FROM relations_tags
        WHERE relations_tags.key="name"

        ) AS all_tables
)
GROUP By ftype_table.value
ORDER BY Total DESC
"""

df_query(query)

,id,value,Total
0,7077413,LakePond,21686
1,7077411,SwampMarsh,16188
2,2313282,StreamRiver,11758
...,...,...,...
10,43275245,Pipeline,6
11,368947130,Wall,6
12,97140992,Lock Chamber,5


In [ ]:
conn.close()

### Consider how the data could be improved:

Investigate (programmatically) the date the data was uploaded, then compare to the date when the NHD switched over to the Permanent identifier

All elements where NHD:GNIS_ID has a duplicate value with gnis:feature_id:

In [18]:
query = """
SELECT
    x.id,
    x.key,
    x.value,
    y.key,
    y.value
FROM (
    SELECT
        *
    FROM nodes_tags
    WHERE nodes_tags.key="GNIS_ID"

    UNION ALL
    
    SELECT
        *
    FROM ways_tags
    WHERE ways_tags.key="GNIS_ID"

    UNION ALL
    
    SELECT
        *
    FROM relations_tags
    WHERE relations_tags.key="GNIS_ID"
) AS x
    JOIN (
    SELECT
    *
    FROM nodes_tags
    WHERE nodes_tags.key="feature_id"
    
    UNION ALL
    
    SELECT
    *
    FROM ways_tags
    WHERE ways_tags.key="feature_id"
    
    UNION ALL
    
    SELECT
    *
    FROM relations_tags
    WHERE relations_tags.key="feature_id"
    ) AS y
    ON x.id=y.id
WHERE x.value=y.value
"""

df_query(query)

,id,key,value,key,value
0,558006771,GNIS_ID,00559711,feature_id,00559711
1,3301246063,GNIS_ID,00554877,feature_id,00554877
2,22516599,GNIS_ID,00538745,feature_id,00538745
...,...,...,...,...,...
924,4626652,GNIS_ID,00532593,feature_id,00532593
925,4626653,GNIS_ID,00555004,feature_id,00555004
926,5380366,GNIS_ID,00532757,feature_id,00532757


All elements where NHD:Permanent_ is duplicate value with NHD:ComID

In [19]:
query = """
SELECT
    x.id,
    x.key,
    x.value,
    y.key,
    y.value
FROM (
    SELECT
        *
    FROM nodes_tags
    WHERE nodes_tags.key="ComID"

    UNION ALL
    
    SELECT
        *
    FROM ways_tags
    WHERE ways_tags.key="ComID"

    UNION ALL
    
    SELECT
        *
    FROM relations_tags
    WHERE relations_tags.key="ComID"
) AS x
    JOIN (
    SELECT
    *
    FROM nodes_tags
    WHERE nodes_tags.key="Permanent_"
    
    UNION ALL
    
    SELECT
    *
    FROM ways_tags
    WHERE ways_tags.key="Permanent_"
    
    UNION ALL
    
    SELECT
    *
    FROM relations_tags
    WHERE relations_tags.key="Permanent_"
    ) AS y
    ON x.id=y.id
WHERE x.value=y.value
"""

df_query(query)

,id,key,value,key,value
0,43239879,ComID,148751000,Permanent_,148751000
1,43244825,ComID,148750992,Permanent_,148750992
2,43246802,ComID,151098269,Permanent_,151098269
...,...,...,...,...,...
1292,4493454,ComID,151099663,Permanent_,151099663
1293,4493455,ComID,151099653,Permanent_,151099653
1294,4493460,ComID,151097798,Permanent_,151097798


NHD:way_id duplicated with NHD:ComID

In [20]:
query = """
SELECT
    x.id,
    x.key,
    x.value,
    y.key,
    y.value
FROM (
    SELECT
        *
    FROM nodes_tags
    WHERE nodes_tags.key="ComID"

    UNION ALL
    
    SELECT
        *
    FROM ways_tags
    WHERE ways_tags.key="ComID"

    UNION ALL
    
    SELECT
        *
    FROM relations_tags
    WHERE relations_tags.key="ComID"
) AS x
    JOIN (
    SELECT
    *
    FROM nodes_tags
    WHERE nodes_tags.key="way_id"
    
    UNION ALL
    
    SELECT
    *
    FROM ways_tags
    WHERE ways_tags.key="way_id"
    
    UNION ALL
    
    SELECT
    *
    FROM relations_tags
    WHERE relations_tags.key="way_id"
    ) AS y
    ON x.id=y.id
WHERE x.value=y.value
"""

df_query(query)

,id,key,value,key,value
0,41225956,ComID,139142734,way_id,139142734
1,41226082,ComID,139142736,way_id,139142736
2,43166285,ComID,148743907,way_id,148743907
...,...,...,...,...,...
12980,481376674,ComID,148740191,way_id,148740191
12981,482476279,ComID,148740240,way_id,148740240
12982,482476281,ComID,148740240,way_id,148740240
